In [ ]:
!pip -q install tytan

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 MB 11.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
from tytan import *

In [ ]:
# とりあえず、指数部なしでquboを作ってみる
def create_float_qubo(dim: int, upper: float, lower: float, num_bit: int):
    qbits = symbols_list([dim, num_bit], 'q{}_{}')
    qfloat = []

    norm = 1.0/(2**num_bit)

    for row in range(dim):
      qf = 0
      for col in range(num_bit):
        qf += qbits[row][col] * (2**col)
      qfloat.append(lower + (upper-lower)*qf*norm)

    return qfloat

In [ ]:
# 指数部も作る
# m_upper, m_lower は仮数部だけの最大、最小
# 10^digits_upper から 10^digits_lower の範囲の指数部
# 次元が多くなるので、Hoboで計算すること
def create_double_qubo(dim: int, m_upper: float, m_lower: float, num_m_bit: int, digits_upper: int, digits_lower: int):
  m_bits = symbols_list([dim, num_m_bit], 'm{}_{}')
  e_bits = symbols_list([dim, digits_upper-digits_lower+1], 'e{}_{}')

  qdouble = []

  Honehot = 0

  # 仮数部を1.0に
  norm = 1.0/(2**num_m_bit)

  for row in range(dim):
    # 仮数部
    qm = 0
    for col in range(num_m_bit):
      qm += m_bits[row][col] * (2**col)
    qmantissa = lower + (upper-lower)*qm*norm

    # 指数部
    qe = 0
    ho = 0
    for col, ex in enumerate(range(digits_lower, digits_upper+1)):
      qe += e_bits[row][col] * (10**ex)
      ho += e_bits[row][col] # ワンホット

    #print('指数 H',qe)

    qdouble.append(qmantissa*qe)
    Honehot += (ho -1)**2

  return qdouble, Honehot

In [ ]:
# 方程式を組んで、そのハミルトニアンを返す
def create_equation_hamiltonian(A, b, qfloat):
  dim = len(b)

  H = 0

  for row in range(dim):
    H_i = 0
    for col in range(dim):
      H_i += A[row][col] * qfloat[col]
    H_i -= b[row]
    H += H_i**2

  return H

In [ ]:
dim = 3
bitnum = 4
upper = 2.0
lower = -2.0
d_upper = 1
d_lower = -1

#qfloat = create_float_qubo(dim, upper, lower, bitnum)
qfloat, Honehot = create_double_qubo(dim, upper, lower, bitnum, d_upper, d_lower)

#print(qfloat)

# 連立方程式
A = np.array([[2, 1, 1],
              [2, 3, 1],
              [1, 1, 3]])
#b = np.array([2, 4, -1])
#b = np.array([2.2, 4.2, -0.4]) # 4bit 1..-1 最終解  [ 1.    1.   -0.75]
b = np.array([23.09, 25.29, 12.07]) # 11, 1.1, -0.01

H = create_equation_hamiltonian(A, b, qfloat)
H += Honehot
#print('全体H ',H)

# solve
qubo, offset = Compile(H).get_hobo()
solver = sampler.ArminSampler(seed=1)
result = solver.run(qubo, shots=100)

# result
norm = 1.0 / (2**bitnum)
print('offset ', offset)
qbest = np.array([0.,0.,0.])
count = 0
for r in result:
  print(r)

  elist = list(r[0].values())[:dim*(d_upper - d_lower +1)]
  mlist = list(r[0].values())[dim*(d_upper - d_lower +1):]

  earray = np.array(elist).reshape([dim,-1])
  marray = np.array(mlist).reshape([dim,-1])

  edigits = np.array([10**i for i in range(d_lower, d_upper+1)])
  mbinary = np.array([2**i for i in range(bitnum)])

  epart = earray @ edigits
  mpart = lower + (upper - lower) * norm *( marray @ mbinary )

  #print('指数部数値 ', epart)
  #print('仮数部数値', mpart)

  xq = mpart * epart

  print('qubo/hoboの解 ', np.array2string(xq, separator=','))
  if count == 0:
    qbest = xq
  count += 1

MODE: GPU
DEVICE: cuda:0
offset  1321.4171
[{'e0_0': 0, 'e0_1': 1, 'e0_2': 1, 'e1_0': 0, 'e1_1': 1, 'e1_2': 0, 'e2_0': 1, 'e2_1': 0, 'e2_2': 0, 'm0_0': 0, 'm0_1': 0, 'm0_2': 1, 'm0_3': 1, 'm1_0': 0, 'm1_1': 0, 'm1_2': 1, 'm1_3': 1, 'm2_0': 0, 'm2_1': 1, 'm2_2': 0, 'm2_3': 1}, -1320.3511, 29]
qubo/hoboの解  [11.  , 1.  , 0.05]
[{'e0_0': 0, 'e0_1': 1, 'e0_2': 1, 'e1_0': 0, 'e1_1': 1, 'e1_2': 0, 'e2_0': 1, 'e2_1': 0, 'e2_2': 0, 'm0_0': 0, 'm0_1': 0, 'm0_2': 1, 'm0_3': 1, 'm1_0': 0, 'm1_1': 0, 'm1_2': 1, 'm1_3': 1, 'm2_0': 0, 'm2_1': 0, 'm2_2': 1, 'm2_3': 1}, -1320.3276, 12]
qubo/hoboの解  [11. , 1. , 0.1]
[{'e0_0': 0, 'e0_1': 1, 'e0_2': 1, 'e1_0': 0, 'e1_1': 1, 'e1_2': 0, 'e2_0': 0, 'e2_1': 1, 'e2_2': 0, 'm0_0': 0, 'm0_1': 0, 'm0_2': 1, 'm0_3': 1, 'm1_0': 0, 'm1_1': 0, 'm1_2': 1, 'm1_3': 1, 'm2_0': 0, 'm2_1': 0, 'm2_2': 0, 'm2_3': 1}, -1320.3203, 11]
qubo/hoboの解  [11., 1., 0.]
[{'e0_0': 0, 'e0_1': 0, 'e0_2': 1, 'e1_0': 0, 'e1_1': 1, 'e1_2': 0, 'e2_0': 1, 'e2_1': 0, 'e2_2': 0, 'm0_0': 0, 'm0_1

In [ ]:
# 参照
# https://watlab-blog.com/2022/09/06/jacobi-method/
def jacobi(A, b, x0, tolerance):
    ''' 連立方程式をヤコビ法（反復法）で解く '''

    # 初期化（適当な解と残差）
    x_1 = x0
    residual = 1e20

    # A = D + R
    D = np.diag(A)
    R = A - np.diag(D)

    # 反復計算=>残差がトレランスより小さくなったら終了
    i = 0
    print('----------Start iteration----------')
    while residual > tolerance:
        # 解と残差を計算
        x = (b - (R @ x_1)) / D
        residual = np.sum(np.sqrt((x - x_1) ** 2)) / np.sum(np.sqrt(x ** 2))

        if i % 100 == 0:
            print('Iteration=', i)
            print('Residual=', residual)
            print('x=', x)
        i += 1
        x_1 = x

    print('----------End iteration----------')
    print('Iteration=', i)
    print('Residual=', residual)
    print('x=', x, '\n')

    return x

In [ ]:
# 係数行列と定数ベクトル

x0 = np.array([0]*len(b))

# ヤコビ法で解を反復計算
x_jacobi = jacobi(A, b, x0, 1e-6)
print('Jacobi method x=', x_jacobi, 'Validated b=', A @ x_jacobi)
print('\n')

# qubo初期値付き
print('qubo/hobo初期値 ', np.array2string(qbest, separator=','))
x_qubo = jacobi(A, b, qbest, 1e-6)
print('QUBO + Jacobi method x=', x_qubo, 'Validated b=', A @ x_jacobi)
print('\n')

# numpyの線形代数ライブラリで検証
x_linalg = np.linalg.solve(A, b)
print('np.linalg.solve x=', x_linalg, 'Validated b=', A @ x_linalg)

----------Start iteration----------
Iteration= 0
Residual= 1.0
x= [11.545       8.43        4.02333333]
Iteration= 100
Residual= 2.4860536705960127e-05
x= [ 1.10000510e+01  1.10005246e+00 -9.96139768e-03]
----------End iteration----------
Iteration= 130
Residual= 9.382489875524209e-07
x= [ 1.09999981e+01  1.09999802e+00 -1.00014569e-02] 

Jacobi method x= [ 1.09999981e+01  1.09999802e+00 -1.00014569e-02] Validated b= [23.08999272 25.28998876 12.06999173]


qubo/hobo初期値  [11.  , 1.  , 0.05]
----------Start iteration----------
Iteration= 0
Residual= 0.010448171569975162
x= [11.02        1.08        0.02333333]
----------End iteration----------
Iteration= 78
Residual= 9.97735925994685e-07
x= [ 1.09999980e+01  1.09999789e+00 -1.00015492e-02] 

QUBO + Jacobi method x= [ 1.09999980e+01  1.09999789e+00 -1.00015492e-02] Validated b= [23.08999272 25.28998876 12.06999173]


np.linalg.solve x= [ 1.1e+01  1.1e+00 -1.0e-02] Validated b= [23.09 25.29 12.07]
